### Import Libraries

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

### Import Dataset

In [2]:
url = "http://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data"
columns = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'class']
df = pd.read_csv(url, names=columns)
df.head()

,buying,maint,doors,persons,lug_boot,safety,class
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


### Preprocessing

In [3]:
# Convert categorical variables to numerical
df_encoded = pd.get_dummies(df, columns=['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety'])

# Features and target variable
X = df_encoded.drop('class', axis=1)
y = df_encoded['class']

# Label encoding for the target variable
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

### 1. Apply XGBoost classifier

In [4]:
# XGBoost classifier
xgb_classifier = XGBClassifier(objective='multi:softmax')
xgb_classifier.fit(X_train, y_train)
y_pred_xgb = xgb_classifier.predict(X_test)

# Print classification report
print("Classification Report for XGBoost:")
print(classification_report(y_test, y_pred_xgb))

Classification Report for XGBoost:
              precision    recall  f1-score   support

           0       1.00      0.92      0.96        83
           1       0.61      1.00      0.76        11
           2       1.00      1.00      1.00       235
           3       1.00      1.00      1.00        17

    accuracy                           0.98       346
   macro avg       0.90      0.98      0.93       346
weighted avg       0.99      0.98      0.98       346



### 2. Apply CatBoost classifier

In [5]:
# CatBoost classifier
cat_classifier = CatBoostClassifier(iterations=150, depth=5, learning_rate=0.3, loss_function='MultiClass', verbose=15)
cat_classifier.fit(X_train, y_train)
y_pred_cat = cat_classifier.predict(X_test)

# Print classification report
print("Classification Report for CatBoost:")
print(classification_report(y_test, y_pred_cat))

0:	learn: 0.9474975	total: 149ms	remaining: 22.1s
15:	learn: 0.2026547	total: 193ms	remaining: 1.62s
30:	learn: 0.1136741	total: 230ms	remaining: 882ms
45:	learn: 0.0760837	total: 267ms	remaining: 603ms
60:	learn: 0.0548474	total: 302ms	remaining: 441ms
75:	learn: 0.0425822	total: 336ms	remaining: 327ms
90:	learn: 0.0346446	total: 375ms	remaining: 243ms
105:	learn: 0.0287692	total: 412ms	remaining: 171ms
120:	learn: 0.0247338	total: 445ms	remaining: 107ms
135:	learn: 0.0219709	total: 479ms	remaining: 49.3ms
149:	learn: 0.0195640	total: 510ms	remaining: 0us
Classification Report for CatBoost:
              precision    recall  f1-score   support

           0       1.00      0.92      0.96        83
           1       0.59      0.91      0.71        11
           2       1.00      1.00      1.00       235
           3       0.94      1.00      0.97        17

    accuracy                           0.98       346
   macro avg       0.88      0.96      0.91       346
weighted avg       0.

### 3. Compare the Overall Accuracy

In [6]:
# Compare accuracy
accuracy_xgb = (y_test == y_pred_xgb).mean()
accuracy_cat = (y_test == y_pred_cat).mean()

print(f"Overall Accuracy - XGBoost: {accuracy_xgb}")
print(f"Overall Accuracy - CatBoost: {accuracy_cat}")

Overall Accuracy - XGBoost: 0.9797687861271677
Overall Accuracy - CatBoost: 0.518109525877911
